In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s6e2/sample_submission.csv
/kaggle/input/playground-series-s6e2/train.csv
/kaggle/input/playground-series-s6e2/test.csv


In [2]:
import pandas as pd

train = pd.read_csv("/kaggle/input/playground-series-s6e2/train.csv")
test = pd.read_csv("/kaggle/input/playground-series-s6e2/test.csv")

train.head()


,id,Age,Sex,Chest pain type,BP,Cholesterol,FBS over 120,EKG results,Max HR,Exercise angina,ST depression,Slope of ST,Number of vessels fluro,Thallium,Heart Disease
0,0,58,1,4,152,239,0,0,158,1,3.6,2,2,7,Presence
1,1,52,1,1,125,325,0,2,171,0,0.0,1,0,3,Absence
2,2,56,0,2,160,188,0,2,151,0,0.0,1,0,3,Absence
3,3,44,0,3,134,229,0,2,150,0,1.0,2,0,3,Absence
4,4,58,1,4,140,234,0,2,125,1,3.8,2,3,3,Presence


In [3]:
train.shape
train.info()
train.describe()
train.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 630000 entries, 0 to 629999
Data columns (total 15 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   id                       630000 non-null  int64  
 1   Age                      630000 non-null  int64  
 2   Sex                      630000 non-null  int64  
 3   Chest pain type          630000 non-null  int64  
 4   BP                       630000 non-null  int64  
 5   Cholesterol              630000 non-null  int64  
 6   FBS over 120             630000 non-null  int64  
 7   EKG results              630000 non-null  int64  
 8   Max HR                   630000 non-null  int64  
 9   Exercise angina          630000 non-null  int64  
 10  ST depression            630000 non-null  float64
 11  Slope of ST              630000 non-null  int64  
 12  Number of vessels fluro  630000 non-null  int64  
 13  Thallium                 630000 non-null  int64  
 14  Hear

id                         0
Age                        0
Sex                        0
Chest pain type            0
BP                         0
Cholesterol                0
FBS over 120               0
EKG results                0
Max HR                     0
Exercise angina            0
ST depression              0
Slope of ST                0
Number of vessels fluro    0
Thallium                   0
Heart Disease              0
dtype: int64

In [4]:
train["Heart Disease"] = train["Heart Disease"].str.strip()
train["Heart Disease"].unique()


array(['Presence', 'Absence'], dtype=object)

In [5]:
train["Heart Disease"] = train["Heart Disease"].map({"Absence":0, "Presence":1})
train["Heart Disease"].dtype


dtype('int64')

In [6]:
train["Heart Disease"].value_counts()


Heart Disease
0    347546
1    282454
Name: count, dtype: int64

In [7]:
train["Heart Disease"].value_counts(normalize=True)


Heart Disease
0    0.55166
1    0.44834
Name: proportion, dtype: float64

In [8]:
train = train.drop("id", axis=1)


In [9]:
X = train.drop("Heart Disease", axis=1)
y = train["Heart Disease"]
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


In [10]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)


In [11]:
from xgboost import XGBClassifier

xgb = XGBClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1,
    eval_metric='logloss'
)

xgb.fit(X_train, y_train)

xgb_pred = xgb.predict(X_val)
xgb_probs = xgb.predict_proba(X_val)[:,1]


In [12]:
from sklearn.metrics import classification_report, roc_auc_score

print(classification_report(y_val, xgb_pred))
print("ROC-AUC:", roc_auc_score(y_val, xgb_probs))


              precision    recall  f1-score   support

           0       0.89      0.91      0.90     69509
           1       0.88      0.87      0.88     56491

    accuracy                           0.89    126000
   macro avg       0.89      0.89      0.89    126000
weighted avg       0.89      0.89      0.89    126000

ROC-AUC: 0.9559488630416589


In [13]:
import pandas as pd

# Load test and sample submission
test = pd.read_csv("/kaggle/input/playground-series-s6e2/test.csv")
sample_sub = pd.read_csv("/kaggle/input/playground-series-s6e2/sample_submission.csv")

# Save ID
test_id = test["id"]

# Drop id column
test = test.drop("id", axis=1)


In [14]:
# Predict probabilities
test_probs = xgb.predict_proba(test)[:,1]

# Convert to binary predictions (default threshold 0.5)
test_preds = (test_probs > 0.5).astype(int)


submission = pd.DataFrame({
    "id": test_id,
    "Heart Disease": test_preds
})

submission.to_csv("submission.csv", index=False)
